In [1]:
import numpy as np
import skimage
import utils
import pathlib

In [ ]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    
    # Compute normalized histogram
    hist, bins = np.histogram(im.flatten(), bins=256, range=(0, 256))
    hist = hist / hist.sum()
    
    # Compute cumulative sums
    cumulative_sum = np.cumsum(hist)
    cumulative_mean = np.cumsum(hist * np.arange(256))
    global_mean = cumulative_mean[-1]    
    
    max_variance = 0
    optimal_threshold = 0
    
    
    # Iterate over all possible thresholds (0–255)
    for threshold in range(256):
        
        P1 = cumulative_sum[threshold]  # Probability background
        P2 = 1 - P1 # Prob background
        
        if P1 == 0 or P2 == 0:
            continue
        
        # means of intensity of the background and foreground
        mean1 = cumulative_mean[threshold] / P1  
        mean2 = (global_mean - cumulative_mean[threshold]) / P2  
        
        # Between-class variance
        between_class_variance = P1 * P2 * (mean1 - mean2) ** 2
        
        # Update the optimal threshold if the between-class variance is maximized
        if between_class_variance > max_variance:
            max_variance = between_class_variance
            optimal_threshold = threshold
    
    return optimal_threshold
    
    

In [4]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

        


Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed\rice-shaded-segmented.png
